1. 엑셀 불러와서 sheet2 만들기

pd.read_excel 옵션값
https://rfriend.tistory.com/464

dataframe 옵션
https://brunch.co.kr/@parkean/7

In [1]:
import pandas as pd
from openpyxl import load_workbook
from datetime import datetime

In [4]:
#이름 입력하도록 설정
path = r"example_file_toyota.xlsm"


In [6]:
data = pd.read_excel(path, sheet_name = 0, na_values= 'NaN',index_col = 'Respondent.Serial') #paht 위치에 있는 엑셀 파일 읽어오기


In [7]:
test_data = data
test_data.head()

,Respondent.Serial.SourceFile,Respondent.Origin,Respondent.Origin.Other,Respondent.ID,DataCollection.Status,DataCollection.InterviewerID,DataCollection.StartTime,DataCollection.FinishTime,DataCollection.MetadataVersionNumber,DataCollection.MetadataVersionGUID,...,Timings.timeTotalMinutesText,Timings.timeTotalMinutes,Satisfaction.q1,Satisfaction.q2,comp,GOADtest,EA0xordxtext,A10x1ord,yyyymmdd,weekno
Respondent.Serial,,,,,,,,,,,,,,,,,,,,,
4,NaN,{htmlplayer},NaN,GEN24_6092006,"{scriptstopped,signal}",NaN,2020-06-23 07:21:49,2020-06-23 07:22:23,1,58a5897c-49f5-45fe-a834-86cb730e4664,...,NaN,NaN,NaN,NaN,{scr_agree},NaN,NaN,NaN,20200623,202026
5,NaN,{htmlplayer},NaN,GEN24_6070007,"{scriptstopped,signal}",NaN,2020-06-23 07:28:26,2020-06-23 07:29:36,1,58a5897c-49f5-45fe-a834-86cb730e4664,...,NaN,NaN,NaN,NaN,{scr_sq5},NaN,NaN,NaN,20200623,202026
6,NaN,{htmlplayer},NaN,GEN24_6038008,"{scriptstopped,signal}",NaN,2020-06-23 07:29:09,2020-06-23 07:31:56,1,58a5897c-49f5-45fe-a834-86cb730e4664,...,NaN,NaN,NaN,NaN,{scr_sq5},NaN,NaN,NaN,20200623,202026
7,NaN,{htmlplayer},NaN,GEN24_6009009,"{scriptstopped,signal}",NaN,2020-06-23 07:30:55,2020-06-23 07:32:14,1,58a5897c-49f5-45fe-a834-86cb730e4664,...,NaN,NaN,NaN,NaN,{scr_sq5},NaN,NaN,NaN,20200623,202026
8,NaN,{htmlplayer},NaN,GEN24_6017010,"{scriptstopped,signal}",NaN,2020-06-23 07:32:35,2020-06-23 07:33:26,1,58a5897c-49f5-45fe-a834-86cb730e4664,...,NaN,NaN,NaN,NaN,{scr_sq3},NaN,NaN,NaN,20200623,202026


In [ ]:
#형태 알아보기

# data.index
# data.dtypes

In [ ]:
#중복있는 값은 True로 체크
data['duplicate_chk'] = (data.duplicated(['IntroStart.IP']))

#확인
test_data_du = data[data['duplicate_chk']==True]
test_data_du.head()

In [ ]:
ip_all =list(data['IntroStart.IP'])
ip_realid = data[['realid','IntroStart.IP']]
# display(ip_realid.head())

ip_unique_list = list((set(ip_all)))

ip_dict = {}
for i in ip_unique_list:
    ip_dict[i] = ip_all.count(i)   

    
ip_dataFrame = pd.DataFrame(list(ip_dict.items()), columns=['IntroStart.IP', 'IP_Count']).sort_values(by='IP_Count',ascending=False)
ip_dataFrame = ip_dataFrame.dropna()
#display(ip_dataFrame.head())


ip_final_table = pd.merge(ip_dataFrame,ip_realid, on='IntroStart.IP', how='left')



In [ ]:
need_columns = ['TotalTime2','realid','comp','IntroStart.IP','duplicate_chk','panelDetails.sampsrc','DeviceDetails.isMobile','projectDetails.version','DataCollection.EndQuestion']

list_data = data[need_columns]
display(list_data)

In [ ]:
list_data.loc[((list_data.comp.str.startswith("{scr_")) | (list_data.comp.str.startswith("{jwquality"))) , "comp"] = "{scr_out}"
list_data

In [ ]:
#margin의 의미는 모든 값을 합산한 값을 보여주는 인덱스를 생성할 지를 결정하는 인자다.

pivot = pd.pivot_table(list_data, index='comp', values ='realid',aggfunc = 'count', margins = True)
pivotdf = pd.DataFrame(pivot)


pivotdf.loc['LOI(min)'] = list_data['TotalTime2'].median()
pivotdf

#부스터 있을 때 각각 계산 하는거 추가해야함


In [ ]:
# 1. 엑셀 파일 열기 w/ExcelWriter
datestring = datetime.strftime(datetime.now(), '%Y%m%d') #저장 날짜로 파일 이름 만들기
filename = 'Respondent_list'+ '_' + datestring.strip() + '_final.xlsx'

# df_needs_columns.to_excel(excel_writer=filename)   
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# 2. 시트별 데이터 추가하기(1개 시트로 저장할때와 유사)
pivotdf.to_excel(writer, sheet_name= 'Pivot_Table')
list_data.to_excel(writer, sheet_name= 'Data')
ip_final_table.to_excel(writer, sheet_name='IP 중복값 Pivot')


# 3. 엑셀 파일 저장하기
writer.save()


In [ ]:
#추가 ID 중복 확인